# Session

Session 은 우리가 데이터베이스로 부터 가져온 **엔티티(Entity)** 들을 **특정 태스크 혹은 연속적인 오퍼레이션**을 지속하는 동안 하나의 스코프로써 사용하기 위해 이용됩니다. 
살짝은 어려울수도 있으니 예시와 함께 이해해보도록 합시다. 이걸 이해하기 위해서는 우리가 **ORM(Object Relational Mapping)** 을 지원해주는 라이브러리를 왜 사용하는지 이해하는 것이 필요합니다. 

우리는 코드상에서 직접 Query 를 통해 데이터의 값을 변경하기 보다는 데이터베이스에서 값을 가져와서 하나의 프록시 객체로 여겨지는 객체를 만들고, 해당 객체체의 메소드를 이용 혹은 연산을 통해 
프로그램 안에서 Object 의 값을 변경 하려고 합니다. 이는 외부 데이터베이스가 무엇인지에 대한 의존성을 줄일 뿐 아니라, 객체 중심으로 상태값을 조작하므로 프로그램을 작성하는 개발자에게 객체 중심적인 개발을 할수 있도록 도와줍니다. 
보통 이렇게 객체화 한뒤, 프로그래밍 내에서 **변경을 추적하고 반영하는 패턴**을 (Unit of Work)[https://martinfowler.com/eaaCatalog/unitOfWork.html] 라고 하는데요.

Session 에서도 우리가 가져온 데이터를 객체화 하고, `identity map` 에 저장하여 관리할 수 있도록 합니다. 그 후, Identity Map 객체 내의 메소드를 통해 우리가 객체내부의 상태값을 변경하게 되면 이를 추적하게 됩니다.
간단한 코드와 함께 이해해보도록 합시다.

In [15]:
class User:
    def __init__(self, id: int, name: str, email: str):
        self.id = id
        self.name = name
        self.email = email

    def __repr__(self):
        return f"User(id={self.id}, name='{self.name}', email='{self.email}')"

    def __eq__(self, other):
        if not isinstance(other, User):
            return NotImplemented
        return self.id == other.id

    def __hash__(self):
        return hash(self.id)

In [21]:
from typing import Dict

class Database:
    def __init__(self):
        self.users = {
            1: User(1, "John Doe", "john@example.com"),
            2: User(2, "Jane Doe", "jane@example.com"),
        }

    def fetch_user(self, id: int):
        return self.users.get(id)
    
    def insert(self, entity: User):
        self.users[entity.id] = entity

    def update(self, id: int, changes: Dict[str, any]):
        self.users[id].__dict__.update(changes)

    def delete(self, id: int):
        del self.users[id]

In [41]:
database = Database()

In [30]:
import copy
from typing import Dict, TypeVar, Generic, Set

T = TypeVar('T')

class SimpleSession(Generic[T]):

    def __init__(self):
        # Identity Map: 로드된 객체의 유일성 보장 (ID -> 객체)
        self.identity_map: Dict[int, T] = {}
        # 변경 추적을 위한 집합 (Set)
        self.new_objects: Set[T] = set()
        self.dirty_objects: Set[T] = set()
        self.removed_objects: Set[T] = set()
        # dirty 객체 비교를 위한 원본 상태 저장 (ID -> 원본 객체 복사본)
        self.original_states: Dict[int, T] = {}

    def _add_to_identity_map(self, entity: T):
        """Identity Map에 객체를 추가하고 원본 상태를 저장 (필요시)"""
        if entity.id not in self.identity_map:
            self.identity_map[entity.id] = entity
            # 원본 상태 스냅샷 저장 (dirty 체크용) - 깊은 복사 고려 필요
            try:
                self.original_states[entity.id] = copy.deepcopy(entity)
            except TypeError:
                 # 경고 로깅 또는 다른 복사 전략 사용
                 print(f"Warning: Could not deepcopy entity {entity.id}. Using shallow copy.")
                 self.original_states[entity.id] = copy.copy(entity)

    def get(self, entity_id: int) -> T | None:
        """
        Identity Map에서 객체를 가져옵니다.
        실제 애플리케이션에서는 DB에서 로드하는 로직이 포함됩니다.
        """
        if entity_id in self.identity_map:
            return self.identity_map[entity_id]

        print(f"Simulating DB fetch for entity ID: {entity_id}")
        user_data = database.fetch_user(entity_id)
        if user_data:
            entity = User(id=user_data.id, name=user_data.name, email=user_data.email)
            self._add_to_identity_map(entity)
            return entity
        else:
            return None

    def add(self, entity: T):
        """새 객체를 Unit of Work에 등록합니다 (INSERT 대상)."""
        if entity.id in self.identity_map:
            print(f"Entity {entity.id} is already managed.")
            return

        print(f"Registering NEW: {entity}")
        self._add_to_identity_map(entity)
        self.new_objects.add(entity)
        # 다른 상태 집합에서는 제거 (혹시 모를 중복 상태 방지)
        self.dirty_objects.discard(entity)
        self.removed_objects.discard(entity)

    def remove(self, entity: T):
        """객체를 Unit of Work에서 제거 대상으로 등록합니다 (DELETE 대상)."""
        entity_id = entity.id
        if entity_id not in self.identity_map:
             print(f"Cannot remove entity {entity_id}: Not managed by this session.")
             return

        print(f"Registering REMOVED: {entity}")
        self.removed_objects.add(entity)
        # 다른 상태 집합에서는 제거
        self.new_objects.discard(entity)
        self.dirty_objects.discard(entity)

    def _detect_dirty_objects(self):
        """
        (선택적) Identity Map의 객체와 원본 상태를 비교하여 자동으로 dirty 상태를 감지합니다.
        명시적인 register_dirty 호출 대신 사용할 수 있지만, 객체 비교 로직이 중요합니다.
        """
        print("Detecting dirty objects...")
        for entity_id, current_entity in self.identity_map.items():
            # new 이거나 removed 된 객체는 dirty 검사 대상이 아님
            if current_entity in self.new_objects or current_entity in self.removed_objects:
                continue

            original_entity = self.original_states.get(entity_id)
            if original_entity:
                # 객체 비교 로직: 실제로는 속성별 비교가 필요할 수 있음
                # 여기서는 간단히 __dict__ 비교 (주의: 모든 경우에 적합하지 않을 수 있음)
                if current_entity.__dict__ != original_entity.__dict__:
                     print(f"Detected DIRTY (via diff): {current_entity}")
                     self.dirty_objects.add(current_entity)
            # else: 원본 상태가 없으면 자동 감지 불가

    def commit(self):
        """변경 사항을 데이터베이스에 영속화합니다."""
        print("\n--- Starting Commit ---")

        # 변경감지 수행
        self._detect_dirty_objects()

        # 실제 DB 작업 수행 (순서 중요: 보통 INSERT -> UPDATE -> DELETE)
        self._insert_new()
        self._update_dirty()
        self._delete_removed()

        # Commit 후 상태 초기화
        self.new_objects.clear()
        self.dirty_objects.clear()
        self.removed_objects.clear()
        self.original_states.clear() # 원본 상태도 클리어 (다음 트랜잭션을 위해)
        # Identity Map은 계속 유지될 수 있음 (세션/UoW 생명주기에 따라)
        # self.identity_map.clear() # 필요하다면 클리어 => 일단은 예시니까 복잡한 상태관리의 디테일까지는 X

        print("--- Commit Finished ---\n")

    def rollback(self):
        """변경 사항을 취소하고 추적 상태를 초기화합니다."""
        print("\n--- Starting Rollback ---")
        # DB 작업은 수행하지 않음
        self.new_objects.clear()
        self.dirty_objects.clear()
        self.removed_objects.clear()
        # Identity Map의 객체들을 원본 상태로 되돌릴 수도 있음 (더 복잡)
        # self.identity_map = copy.deepcopy(self.original_states) # 간단한 복원 시도
        self.original_states.clear()
        # Identity Map 자체를 클리어할 수도 있음
        # self.identity_map.clear()
        print("--- Rollback Finished ---\n")

    # --- Private Helper Methods for DB Interaction (Simulation) ---

    def _insert_new(self):
        """새 객체들을 데이터베이스에 INSERT 합니다."""
        print(f"INSERTING {len(self.new_objects)} new object(s):")
        for entity in list(self.new_objects): # 순회 중 변경될 수 있으므로 리스트 복사
            print(f"  INSERT INTO {type(entity).__name__} ({', '.join(entity.__dict__.keys())}) VALUES (...) for {entity}")
            # 실제 DB INSERT 로직
            database.insert(entity)

            # 성공적으로 INSERT 후, 객체는 더 이상 'new'가 아님
            # identity_map에는 이미 있으므로, 원본 상태를 업데이트 할 수 있음
            try:
                self.original_states[entity.id] = copy.deepcopy(entity)
            except TypeError:
                 print(f"Warning: Could not deepcopy entity {entity.id} after insert. Using shallow copy.")
                 self.original_states[entity.id] = copy.copy(entity)


    def _update_dirty(self):
        """변경된 객체들을 데이터베이스에 UPDATE 합니다."""
        print(f"UPDATING {len(self.dirty_objects)} dirty object(s):")
        for entity in list(self.dirty_objects):
            original = self.original_states.get(entity.id)
            if original:
                # 변경된 필드만 찾아 업데이트 (더 효율적)
                changes = self._diff_entities(original, entity)
                if changes:
                    print(f"  UPDATE {type(entity).__name__} SET {changes} WHERE id={entity.id} (Original: {original})")
                    # 실제 DB UPDATE 로직
                    database.update(entity.id, changes)
                else:
                     print(f"  Skipping UPDATE for {entity.id}: No changes detected.")
            else:
                # 원본이 없으면 전체 필드 업데이트 또는 에러 처리
                print(f"  UPDATE {type(entity).__name__} SET ... WHERE id={entity.id} (Warning: No original state for diff)")
                # 실제 DB UPDATE 로직 (모든 필드)
                database.update_all_fields(entity)

            # 성공적으로 UPDATE 후, 객체는 더 이상 'dirty'가 아님
            # 원본 상태를 현재 상태로 업데이트
            try:
                self.original_states[entity.id] = copy.deepcopy(entity)
            except TypeError:
                print(f"Warning: Could not deepcopy entity {entity.id} after update. Using shallow copy.")
                self.original_states[entity.id] = copy.copy(entity)


    def _delete_removed(self):
        """삭제 대상으로 표시된 객체들을 데이터베이스에서 DELETE 합니다."""
        print(f"DELETING {len(self.removed_objects)} removed object(s):")
        for entity in list(self.removed_objects):
            print(f"  DELETE FROM {type(entity).__name__} WHERE id={entity.id}")
            # 실제 DB DELETE 로직
            database.delete(entity.id)

            # 성공적으로 DELETE 후, Identity Map 및 원본 상태에서 제거
            entity_id = entity.id
            if entity_id in self.identity_map:
                del self.identity_map[entity_id]
            if entity_id in self.original_states:
                del self.original_states[entity_id]


    def _diff_entities(self, original: T, updated: T) -> Dict[str, any]:
        """두 객체 상태를 비교하여 변경된 속성을 찾아 반환합니다."""
        changes = {}
        if not original or not updated or type(original) is not type(updated):
            return {}

        original_dict = original.__dict__
        updated_dict = updated.__dict__

        all_keys = set(original_dict.keys()) | set(updated_dict.keys())

        for key in all_keys:
            # id 필드는 보통 변경되지 않으므로 제외
            if key == 'id':
                continue

            val_orig = original_dict.get(key, None)
            val_upd = updated_dict.get(key, None)

            if val_orig != val_upd:
                changes[key] = val_upd # 변경된 값을 저장

        return changes

In [42]:
session = SimpleSession[User]()


# 세션을 통해 user2 가져오기
user2 = session.get(2)
if user2:
    print(f"Retrieved from session: {user2}")
    user2.email = "bob.updated@example.com"

session.commit()

# Commit 후 상태 확인
print(f"Identity Map after commit: {session.identity_map}")
print(f"Original States after commit: {session.original_states}")

# 데이터베이스 상태 확인
database.users

Simulating DB fetch for entity ID: 2
Retrieved from session: User(id=2, name='Jane Doe', email='jane@example.com')

--- Starting Commit ---
Detecting dirty objects...
Detected DIRTY (via diff): User(id=2, name='Jane Doe', email='bob.updated@example.com')
INSERTING 0 new object(s):
UPDATING 1 dirty object(s):
  UPDATE User SET {'email': 'bob.updated@example.com'} WHERE id=2 (Original: User(id=2, name='Jane Doe', email='jane@example.com'))
DELETING 0 removed object(s):
--- Commit Finished ---

Identity Map after commit: {2: User(id=2, name='Jane Doe', email='bob.updated@example.com')}
Original States after commit: {}


{1: User(id=1, name='John Doe', email='john@example.com'),
 2: User(id=2, name='Jane Doe', email='bob.updated@example.com')}

In [38]:
session = SimpleSession[User]()

# 세션을 통해 user3 가져오기 및 삭제
user1 = session.get(1)
if user1:
    print(f"Retrieved from session: {user1}")
    session.remove(user1)

# Commit 실행 (DB 작업 시뮬레이션)
session.commit()

# Commit 후 상태 확인
print(f"Identity Map after commit: {session.identity_map}")
print(f"Original States after commit: {session.original_states}")

# 데이터베이스 상태 확인
database.users

Simulating DB fetch for entity ID: 1
Retrieved from session: User(id=1, name='John Doe', email='john@example.com')
Registering REMOVED: User(id=1, name='John Doe', email='john@example.com')

--- Starting Commit ---
Detecting dirty objects...
INSERTING 0 new object(s):
UPDATING 0 dirty object(s):
DELETING 1 removed object(s):
  DELETE FROM User WHERE id=1
--- Commit Finished ---

Identity Map after commit: {}
Original States after commit: {}


{2: User(id=2, name='Jane Doe', email='bob.updated@example.com')}

In [39]:
# 롤백 예시
user4 = User(id=4, name="David", email="david@example.com")
session.add(user4)
user1_retrieved = session.get(1)
if user1_retrieved:
    user1_retrieved.name = "Alice Smith"

session.rollback() # 변경사항 취소

print(f"Identity Map after rollback: {session.identity_map}") # user4는 없을 수 있음 (롤백 정책따라)
print(f"New objects after rollback: {session.new_objects}") # 비어있어야 함
print(f"Dirty objects after rollback: {session.dirty_objects}") # 비어있어야 함

# 커밋 후 데이터베이스 상태 확인
database.users

Registering NEW: User(id=4, name='David', email='david@example.com')
Simulating DB fetch for entity ID: 1

--- Starting Rollback ---
--- Rollback Finished ---

Identity Map after rollback: {4: User(id=4, name='David', email='david@example.com')}
New objects after rollback: set()
Dirty objects after rollback: set()


{2: User(id=2, name='Jane Doe', email='bob.updated@example.com')}

제가 예시로 짠 **Unit of work** 로직이 조금은 어려울 수 있지만 함께 샅샅이 뜯어보도록 합시다. 일단 중요한 부분은 `identity_map` 과 변경 추적을 위한 집합 Set 을 봐주시면 됩니다. 
우리가 객체지향적으로 프로그래밍을 하다보면, 비즈니스 로직을 수행하며 객체의 상태값을 바꾸는 작업들을 많이 하게 됩니다. 예를 들면, 우리의 예시에서는 User 의 이름을 바꾼다거나, 메일을 바꾼다거나 등의 작업이겠죠. 
이러한 작업마다 UPDATE 쿼리를 날리게 된다면, I/O 가 증가하게 되어 더 큰 자원을 소모하게 됩니다. 따라서 변경을 추적하고, 마지막 commit 할때 기준으로 데이터 베이스에 반영하게 되어 **I/O 를 줄이는 일종의 최적화 작업을 진행**합니다. 

```python
class SimpleSession(Generic[T]):

    def __init__(self):
        # Identity Map: 로드된 객체의 유일성 보장 (ID -> 객체)
        self.identity_map: Dict[int, T] = {}
        # 변경 추적을 위한 집합 (Set)
        self.new_objects: Set[T] = set()
        self.dirty_objects: Set[T] = set()
        self.removed_objects: Set[T] = set()
        # dirty 객체 비교를 위한 원본 상태 저장 (ID -> 원본 객체 복사본)
        self.original_states: Dict[int, T] = {}

    def _add_to_identity_map(self, entity: T):
        """Identity Map에 객체를 추가하고 원본 상태를 저장 (필요시)"""
        if entity.id not in self.identity_map:
            self.identity_map[entity.id] = entity
            # 원본 상태 스냅샷 저장 (dirty 체크용) - 깊은 복사 고려 필요
            try:
                self.original_states[entity.id] = copy.deepcopy(entity)
            except TypeError:
                 # 경고 로깅 또는 다른 복사 전략 사용
                 print(f"Warning: Could not deepcopy entity {entity.id}. Using shallow copy.")
                 self.original_states[entity.id] = copy.copy(entity)

    def get(self, entity_id: int) -> T | None:
        """
        Identity Map에서 객체를 가져옵니다.
        실제 애플리케이션에서는 DB에서 로드하는 로직이 포함됩니다.
        """
        if entity_id in self.identity_map:
            return self.identity_map[entity_id]

        print(f"Simulating DB fetch for entity ID: {entity_id}")
        user_data = database.fetch_user(entity_id)
        if user_data:
            entity = User(id=user_data.id, name=user_data.name, email=user_data.email)
            self._add_to_identity_map(entity)
            return entity
        else:
            return None

    def add(self, entity: T):
        """새 객체를 Unit of Work에 등록합니다 (INSERT 대상)."""
        if entity.id in self.identity_map:
            print(f"Entity {entity.id} is already managed.")
            return

        print(f"Registering NEW: {entity}")
        self._add_to_identity_map(entity)
        self.new_objects.add(entity)
        # 다른 상태 집합에서는 제거 (혹시 모를 중복 상태 방지)
        self.dirty_objects.discard(entity)
        self.removed_objects.discard(entity)
```
(**보통, JPA 포함 몇몇 ORM 에서 수정을 `dirty` 라고 표현하기 때문에 코드 내부에서도 dirty 로 표현하였습니다.)

위의 로직을 보시면 `get` 으로 database 에서 fetch 해오는 순간부터 `identity_map` 에 등록되며 그 이후에 객체의 데이터가 변경되었는지를 추적하기 위해 `_add_to_identity_map` 메소드 내부에서 객체의 깊은 복사(얕은 복사는 참조복사이므로 참조하고 있는 객체 변경시 초기기값을 보존할수 없으므로) 를 통해 `original_status` 로써 복사합니다. 

## 변경감지

변경감지 부분은 사실 복잡하게 코딩은 하지 않았고, 최대한 이해하기 쉽게 기능적인 부분만 보여줄 수 있도록 작성하였습니다. 아래 코드를 한번 함께 보시죠.

```python
    def _detect_dirty_objects(self):
        """
        (선택적) Identity Map의 객체와 원본 상태를 비교하여 자동으로 dirty 상태를 감지합니다.
        명시적인 register_dirty 호출 대신 사용할 수 있지만, 객체 비교 로직이 중요합니다.
        """
        print("Detecting dirty objects...")
        for entity_id, current_entity in self.identity_map.items():
            # new 이거나 removed 된 객체는 dirty 검사 대상이 아님
            if current_entity in self.new_objects or current_entity in self.removed_objects:
                continue

            original_entity = self.original_states.get(entity_id)
            if original_entity:
                # 객체 비교 로직: 실제로는 속성별 비교가 필요할 수 있음
                # 여기서는 간단히 __dict__ 비교 (주의: 모든 경우에 적합하지 않을 수 있음)
                if current_entity.__dict__ != original_entity.__dict__:
                     print(f"Detected DIRTY (via diff): {current_entity}")
                     self.dirty_objects.add(current_entity)
            # else: 원본 상태가 없으면 자동 감지 불가

    def commit(self):
        """변경 사항을 데이터베이스에 영속화합니다."""
        print("\n--- Starting Commit ---")

        # 변경감지 수행
        self._detect_dirty_objects()

        # 실제 DB 작업 수행 (순서 중요: 보통 INSERT -> UPDATE -> DELETE)
        self._insert_new()
        self._update_dirty()
        self._delete_removed()

        # Commit 후 상태 초기화
        self.new_objects.clear()
        self.dirty_objects.clear()
        self.removed_objects.clear()
        self.original_states.clear() # 원본 상태도 클리어 (다음 트랜잭션을 위해)
```

(**여기서는 `__dict__` 를 통해 비교했지만, 보통의 프레임워크에서는 객체 자체의 `equals` 나 프레임워크 내부의 변경 방식(전체 비교) 등등의 **비교 우선순위**를 가지고 있습니다)

즁요한 점은 변경감지된 점을 COMMIT 한 시점에 데이터 베이스에 반영하게 되어 실제로 **DATABASE 의 호출횟수**를 줄입니다. 즉, 객체지향 개발자에게 최대한 기존 개발 방식을 고수할수 있게 도와주되 
내부에서 DATABASE 의 접근을 최대한 줄이려고 노력하는 것이죠. 그리고 각 Session 이 커밋될때마다 기존 세션의 데이터를 clear 해주는 작업을 진행하게 되는데, 이는 우리의 비즈니스 로직을 하나의 논리적인 단위로 
인식하고, 해당 논리적 작업에서 발생한 변경을 데이터베이스에 반영할 수 있도록 도와줍니다. (물론 이 부분에서는 프레임워크 부분과 데이터베이스의 논리 트랜잭션 레벨의 개입도 필요합니다)

아래 예시를 보시면 조금 더 이해가 잘 가실꺼라고 생각됩니다.

## 실제 예시

```python
session = SimpleSession[User]()


# 세션을 통해 user2 가져오기
user2 = session.get(2)
if user2:
    print(f"Retrieved from session: {user2}")
    user2.email = "bob.updated@example.com"

session.commit()

# Commit 후 상태 확인
print(f"Identity Map after commit: {session.identity_map}")
print(f"Original States after commit: {session.original_states}")

# 데이터베이스 상태 확인
database.users
```

```sh
Simulating DB fetch for entity ID: 2
Retrieved from session: User(id=2, name='Jane Doe', email='jane@example.com')

--- Starting Commit ---
Detecting dirty objects...
Detected DIRTY (via diff): User(id=2, name='Jane Doe', email='bob.updated@example.com')
INSERTING 0 new object(s):
UPDATING 1 dirty object(s):
  UPDATE User SET {'email': 'bob.updated@example.com'} WHERE id=2 (Original: User(id=2, name='Jane Doe', email='jane@example.com'))
DELETING 0 removed object(s):
--- Commit Finished ---

Identity Map after commit: {2: User(id=2, name='Jane Doe', email='bob.updated@example.com')}
Original States after commit: {}

{1: User(id=1, name='John Doe', email='john@example.com'), 2: User(id=2, name='Jane Doe', email='bob.updated@example.com')}
```

보시면 우리가 코드 내부에서 객체를 변경한 점을 Session 내에서 검출하고, 이를 검사하여 변경되었다고 표기(dirty marking) 합니다. 변경된 점은 데이터 베이스에 반영 된 사실도 확인할 수 있죠. 
객체를 삭제하는 것 또한 잘 동작합니다. 

```python
session = SimpleSession[User]()

# 세션을 통해 user3 가져오기 및 삭제
user1 = session.get(1)
if user1:
    print(f"Retrieved from session: {user1}")
    session.remove(user1)

# Commit 실행 (DB 작업 시뮬레이션)
session.commit()

# Commit 후 상태 확인
print(f"Identity Map after commit: {session.identity_map}")
print(f"Original States after commit: {session.original_states}")

# 데이터베이스 상태 확인
database.users
```

```sh
Simulating DB fetch for entity ID: 1
Retrieved from session: User(id=1, name='John Doe', email='john@example.com')
Registering REMOVED: User(id=1, name='John Doe', email='john@example.com')

--- Starting Commit ---
Detecting dirty objects...
INSERTING 0 new object(s):
UPDATING 0 dirty object(s):
DELETING 1 removed object(s):
  DELETE FROM User WHERE id=1
--- Commit Finished ---

Identity Map after commit: {}
Original States after commit: {}

{2: User(id=2, name='Jane Doe', email='bob.updated@example.com')}
```

## 롤백(Rollback)

롤백또한 잘 동작하는데요. 비즈니스로직내에서 객체의 상태가 예상치 못하게 변경되거나, 들어가서는 안되는 값이 저장되는 경우 보통 **예외처리(raise Exception)**를 하게 됩니다. 
이 경우 데이터베이스에 적재되서는 안되는 상황이죠. 따라서 변경하지 않고 데이터의 변경사항이 **롤백**되어야 합니다.

```python
# 롤백 예시
user4 = User(id=4, name="David", email="david@example.com")
session.add(user4)
user1_retrieved = session.get(1)
if user1_retrieved:
    user1_retrieved.name = "Alice Smith"

session.rollback() # 변경사항 취소

print(f"Identity Map after rollback: {session.identity_map}") # user4는 없을 수 있음 (롤백 정책따라)
print(f"New objects after rollback: {session.new_objects}") # 비어있어야 함
print(f"Dirty objects after rollback: {session.dirty_objects}") # 비어있어야 함

# 커밋 후 데이터베이스 상태 확인
database.users
```

위와 같이 가볍게 구현해 보았는데요. 이제 이 지식을 살려 SQLAlchemy 의 Session 은 어떻게 동작하는지 공부해봅시다.